Plate Girder Design
===================

In [1]:
#@title Initial Beam Data {run: "auto", display-mode: "form" }  
#@markdown Total Length (ft)
span = 40 #@param {type: "number"}
#@markdown Unfactored Dead Load (kips)
DL = 15 #@param {type: "number"}
#@markdown Unfactored Live Load (kips)
LL = 82 #@param {type: "number"}
#@markdown Self Weight Guesstimate (lb/ft)
swg = 100 #@param {type: "number"}
#@markdown LRFD Load Factors
DLF = 1.2 #@param {type: "number"}
LLF = 1.6 #@param {type: "number"}

weight = span*swg/1000

factored_load = (DL*DLF)+(LL*LLF)
total_load = (factored_load*2)+weight

column_reaction = total_load/2

print('Total Load on each end', factored_load, 'kips')
print('Total Load on beam,', total_load, 'kips')
print('Reactions at columns', column_reaction, 'kips')



Total Load on each end 149.20000000000002 kips
Total Load on beam, 302.40000000000003 kips
Reactions at columns 151.20000000000002 kips


In [2]:
#@title Steel Data {run: "auto", display-mode: "form" }  
#@markdown Modulus of Elasticity (ksi)
E = 29000 #@param {type: "number"}
#@markdown Utimate Strength (ksi)
Fu = 65 #@param {type: "number"}
#@markdown Yield Strength (ksi)
Fy = 50 #@param {type: "number"}


## Givens
------
  - Welded Plate Girder
- Type of Steel
  - Do the flange and web use the same steel?
- Are there external supports/bracing (lateral/transverse)?
- What are the specific requirements for the problem?
  - non-compact web section?
  - max height?
  - efficiency requirement?

In [3]:
#@title Static Analysis {run: "auto", display-mode: "form" }  
#@markdown Max Shear (kips)
Vu_max = 150.4 #@param {type: "number"}
#@markdown Max Moment (kips)
Mu_max = 1798 #@param {type: "number"}

## 1. Start with best guess for dimensions

Determine overall depth
  - $d \approx \dfrac{span}{12}$


In [4]:
d1 = span/12
print(d1,'ft')



3.3333333333333335 ft


In [5]:
#@title Adjust depth and convert to inches (manual) {run: "auto", display-mode: "form" }  
d = 32 #@param {type: "number"}

print('Girder Depth', d, 'in')

Girder Depth 32 in


Pick plate thickness to start
  - $t_f = 1" \;\; \scriptsize\text{ (not required to be 1)}$

In [6]:
#@ {run: "auto", display-mode: "form" }  

tf = 1 #@param {type: "number"}

Calculate web height
  - $h = d-2t_f$

In [7]:
h=d-(2*tf)
print('Girder height', h, 'in')

Girder height 30 in


Calculate.... plastification? 
  - $\lambda_{rw} = 5.70 \sqrt{\dfrac{E}{F_y}} \;\; \scriptsize\text{ (case 15)}$ __________________
  - $\lambda_{pw} = 3.76 \sqrt{\dfrac{E}{F_y}} \;\; \scriptsize\text{ (case 15)}$ __________________


In [8]:
import math 
efy = E/Fy
lambda_rw = 5.70*math.sqrt(efy)
lambda_pw = 3.76*math.sqrt(efy)

print('lambda rw', lambda_rw)
print('lambda pw', lambda_pw)

lambda rw 137.27417819823216
lambda pw 90.55279123251806


Determine minimum thickness to be efficient
  - $t_{w.min} = \dfrac{h_w}{\lambda_{rw}}$


In [9]:
twmin = h/lambda_rw
print('Minimum web thickness is %.3f %s' % (twmin,'in'))


Minimum web thickness is 0.219 in


In [10]:
#@title Round up to a multiple of 1/16" {run: "auto", display-mode: "form" }  

tw = .3125 #@param {type: "number"}

f. Calculate necessary plate ratios per use case (Table B4.1b)
  - $\lambda_w = \dfrac{h}{t_w}$
  

In [11]:
lambda_w = h/tw
print('Web slenderness is %.3f' % twmin)

Web slenderness is 0.219


g. Verify non-slender web (if required)
  - $\lambda_p \leq \lambda_w \leq \lambda_r$

  {{lambda_r}}

In [12]:
from IPython.display import Markdown as md

fr=2 #GHz

md("$f_r = %i$ GHz"%(fr))

$f_r = 2$ GHz

from IPython.display import Markdown as md

$f_r = 2$

fr=2 #GHz

md("$f_r = %i$ GHz"%(fr))

h. Determine preliminary flange size using actual $M_{max}$ (from statics analysis)
  - $A_f = \dfrac{M_u}{F_y(h+t_f)} \;\; \scriptsize\text{ Area of flange}$   
  - Divide by $t_f$

In [13]:
bf=14

In [14]:
#@title Round up to a multiple of 1/16" {run: "auto", display-mode: "form" }  

bf = 1 #@param {type: "number"}

i. Think through if ratio, flange width to depth, makes sense

j. Check flange slenderness (Table B4.1b)
  - $\lambda_f = \dfrac{b_f}{2t_f} \leq \lambda_p = 0.38 \sqrt{\dfrac{E}{F_y}} \;\; \scriptsize\text{ (case 11)}$ 

In [15]:
lambda_f = bf/(2*tf)


***Re-proportion if necessary***

Adjust total depth and/or plate thickness

Recalculate $A_f$ then calculate for width
  - $b_f = \dfrac{A_f}{t_f} \;\; \text{ round up}$ __________________

Re-check web slenderness ($\lambda_w$)
  - $\lambda_p \leq \dfrac{h}{t_w} \leq \lambda_r$ __________________
  
Re-check flange slenderness (Table B4.1b)
  - $\lambda_f \leq \lambda_p \;\; \scriptsize\text{ (case 11)}$ __________________

---

In [16]:
#@ {run: "auto", display-mode: "form" }  

tf = 1 #@param {type: "number"}

## Evaluate Trial Section

"Collect" all variables thus far
  - $d \;\scriptsize\text{(total depth, } in.)$
  - $t_w \;\scriptsize\text{(web thickness, } in.)$
  - $h \;\scriptsize\text{(web height, } in.)$
  - $t_f \;\scriptsize\text{(flange thickness, } in.)$
  - $b_f \;\scriptsize\text{(flange width, } in.)$

In [17]:
print('d =',d)
print('tw =', tw)
print('h =', h)
print('tf =', tf)
print('bf =', bf)

d = 32
tw = 0.3125
h = 30
tf = 1
bf = 1



Get other numbers from AutoCad (or different source)
  - $A \;\scriptsize\text{(cross-sectional area, } in.^2)$ __________________ 
  - $I_{xx} \;\scriptsize\text{(moment of inertia about the x-axis, } in.^4)$ __________________ 
  - $r_x \;\scriptsize\text{(radius of gyration about the x-axis, } in.)$ __________________ 
  - $S_{xx} \;\scriptsize\text{(section modulus about the x-axis, } in.^3)$ __________________ 
    - $S_{xx}=I_{xx}/(d/2)$

---

In [18]:
#@title Girder Details from AutoCad {run: "auto", display-mode: "form" }  
#@markdown Area (in2)
A = 39.375 #@param {type: "number"}
#@markdown Moment of Inertia (in4)
Ixx = 7913.125 #@param {type: "number"}
#@markdown Radius of Gyration (in)
rx = 14.176 #@param {type: "number"}
#@markdown Section Modulus (in3) - Calculated
Sxx=Ixx/(d/2)
print('Sxx = ', Sxx)

Sxx =  494.5703125


## 2. Check Flexural Strength of the Girder

$\phi = 0.90$

**Lowest will govern**
    
$\Box$ Yield <br>
$\Box$ Lateral Torsional Buckling <br>
$\Box$ Flange Local Buckling

### **A. Calculate Web Plastification Factor, $R_{pc}$**

1. Calculate necessary variables
   
   - a. Use AutoCAD to find the centroid of the top half ($y_1, in.$) if doubly symmetric then $y_1=y_2$ <br><br> __________________ 

   - b. $Z_{xx} \;\scriptsize\text{(plastic section modulus about the x-axis, } in.^3)$ __________________ 
        - $Z_{xx} = \dfrac{A}{2}(y_1+y_2)$
   - c. $M_p \;\scriptsize\text{(plastic bending moment, } in.^k)$ __________________ 
       - $M_p = Z_{xx}F_y$

   - d. $M_{yc} \;\scriptsize\text{(flexural yield moment for compression, } in.^k)$ __________________ 
       - $M_{yc} = S_{xx}F_y$

2. $R_{pc} = \left[\dfrac{M_p}{M_{yc}} -\left(\dfrac{M_p}{M_{yc}}-1\right)\left(\dfrac{\lambda_w-\lambda_{pw}}{\lambda_{rw}-\lambda_{pw}}\right)\right] \leq \dfrac{M_p}{M_{yc}}$
  <br><br> __________________ $\leq$ __________________ <br>$\scriptsize\text{(lower half governs)}$

### **B. For Flexural Yield**
1. $\phi M_n = \phi R_{pc}F_yS_x
    \;\; \text{\scriptsize (check units of result)}$ __________________ 

2. $\phi M_n \geq M_{max}$ __________________ 

### **C. For Lateral-Torsional Buckling**

1. $r_t \; \scriptsize\text{(effective radius of gyration for LTB; verify shape to determine equation, } in.)$
    - a. $h_o \;\scriptsize\text{(distance between flange centroids, } in.)$ __________________ 
  
    - b. $a_w = h_ot_w/b_{fc}t_{fc}$ __________________ 
  
    - c. $r_t = \dfrac{b_{fc}}{\sqrt{12(1+\frac{1}{6}aw)}}
       \;\; \scriptsize\text{ [F4-11]}$ __________________ 
2. $L_p \;\scriptsize\text{(laterally unbraced length for limit state of yielding, } in.)$ __________________ 
    - a. $L_p = 1.1r_t\sqrt{\dfrac{E}{F_y}}$
3. $L_b \;\scriptsize\text{(distance for each span between brace points, } in.)$
    - a. end-span __________________ 
    - b. midspan __________________ 
    - c. any others? __________________ 

4. Smaller of $L_b$ or $L_p$ is limit state
    - a. $J \;\scriptsize\text{(torsional constant, } in.^4)$ __________________ 
      - $J = \sum \dfrac{1}{3}bt^3
         \;\; \scriptsize\text{ sum of each plate: flange + web + flange}$
    - b. $L_r \;\scriptsize\text{(unbraced length for the limit state of inelastic LTB, }in)$ __________________ 
      - $L_r = 1.95 r_t \dfrac{E}{F_L}
       \sqrt{\dfrac{J}{S_{xc}h_o}+\sqrt{
         \dfrac{J}{S_{xc}h_o}^2+6.76 \left(\dfrac{F_L}{E}\right)^2}}
       \;\; \scriptsize\text{ [F4-8]}$
    - c. Check $\Rightarrow L_p \leq L_b \leq L_r$ __________________ $\;\;$ _________________ 

    - d. $C_b \;\scriptsize\text{(LTB modification factor)}$
      1. $M_{max} \;\scriptsize\text{(max Moment, } in.^k)$ __________________ $\;\;$ _________________ 
   
      2. Absolute Moment value of segment
           - $M_A \;\scriptsize\text{(quarter point)}$ __________________ $\;\;$ _________________ 
           - $M_B \;\scriptsize\text{(centerline)}$ __________________ $\;\;$ _________________ 
           - $M_C \;\scriptsize\text{(third-quarter point)}$ __________________ $\;\;$ _________________ 
  
      3. $C_b = \dfrac{12.5 M_{max}} {2.5M_{max} + 3M_A + 4M_B + 3M_C} \;\; \scriptsize\text{ [F1-1]}$  
           - If $C_b \leq 1.0 \Rightarrow C_b=1.0$
      <br><br>
      __________________ $\;\;$ _________________ 

5. Calculate LTB maximum allowable moment ($\phi M_u$)
  and yield upper limit ($\phi R_{pc} M_{yc}$) for all span sections
    - a. $\scriptsize\text{ [F4-2]}\; \; \normalsize \phi M_n = \phi C_b \left
  [R_{pc} M_{yc}- (R_{pc} M_{yc}-F_L S_{xc})
  \left(\dfrac{L_b-L_p}{L_r-L_p}\right)\right]$ <br><br>__________________ $\;\;$ _________________ 
  <br>
   $\leq$<br>
   $\phi R_{pc} M_{yc}$
   <br><br>__________________ $\;\;$ _________________ 

    - b. Lower number governs
      __________________ $\;\;$ _________________ 
    - c. Check $\Rightarrow$ for all spans $\phi M_n > M_{max}$
      __________________ $\;\;$ _________________ 

### **D. For Compression Flange Local Buckling**

- Since $\lambda_f \leq \lambda_{pf}$, flange is compact. Flange Local Buckling does not apply. $\scriptsize\text{ [F4-3]}$

### **E. For Tension Flange Yielding**

- Since doubly symmetric, this does not apply. (Compression or LTB will govern)

>## ___*Summary for Flexure*___
>@ ______________ span: $\phi M_n$ ______________ $ft^k > M_u$ ______________ $ft^k\;\;\text{  }$ ___________
>@ ______________ span: $\phi M_n$ ______________ $ft^k > M_u$ ______________ $ft^k\;\;\text{  }$ ___________
>@ ______________ span: $\phi M_n$ ______________ $ft^k > M_u$ ______________ $ft^k\;\;\text{  }$ ___________
---

##3. Check Shear Strength of the Girder

$\phi = 0.90$

### **A. Determine Stiffener Spacing $\scriptsize\text{ [AISC G2.2]}$**

- $K_v=5.34 \mid a/h \lt 3.0$
- Max Spacing then...
  - $a \;\scriptsize\text{(clear distance between transverse stiffeners)}$
  - $a \leq (3.0)h$  __________________ 

### **B. Stiffener Spacing**
1. Create distance from end of span __________________ 
    - start with 1-3'?
2. Review remaining spans and evaluate shear __________________ 
    - try to divide high shear areas evenly to remain under max spacing
    - any low shear areas can have larger span, but will require $K_v$ adjusted
3. Calculate $k_v$ for each indivdual span $a$ will change for each span <br><br> 
   _______________________________________________________________ <br><br><br>
    - $k_v \;\scriptsize\text{(web plate shear buckling coefficient)}$
      - $k_v=5.34$
        - without transverse stiffiners
        - $a/h > 3.0$
      - $k_v=5+\dfrac{5}{(a/h)^2} \;\; \scriptsize\text{ [G2-5]}$
      <br><br> ________________ $\;\;$ ________________ $\;\;$ ________________ 

### **C. Determine if Tension Field Action will increase the shear strength**
(sometimes proportions will make TFA not worth it. TFA not allowed for end span or if spacing is greater than the max allowable)
1. $C_{v2} \;\scriptsize\text{(web shear buckling coefficient)}$  __________________ $\;\;$ _________________ 
    - $h/t_w \leq 1.10\sqrt{k_vE/F_y}$
      - TFA will not provide additional shear strength
      - $C_{v2}=1.0
        \;\; \scriptsize\text{ [G2-9]}$
  
    - $1.10\sqrt{k_vE/F_y} < h/t_w \leq 1.37\sqrt{k_vE/F_y}$
      - $C_{v2}= \dfrac{1.10\sqrt{k_vE/F_y}}{h/t_w}
        \;\; \scriptsize\text{ [G2-10]}$

    - $h/t_w>1.37\sqrt{k_vE/F_y}$
      - $C_{v2}=\dfrac{1.51k_vE}{(h/t_w)^2F_y}
        \;\; \scriptsize\text{ [G2-11]}$

2. Web Slenderness ($\lambda_w$) will determine how 
  nominal shear strength ($V_n$) is calculated.
    - a. Calculate Shear Capacity   __________________ $\;\;$ _________________ 
      - $h/t_w \leq 1.10\sqrt{k_vE/F_y}$
        - $\phi V_n= \phi 0.6 F_y A_w
          \;\; \scriptsize\text{ [G2-6]}$

      - If all four requirements are met:
        - $h/t_w > 1.10\sqrt{k_vE/F_y}$
        - $2A_w/(A_{fc}+A_{ft})\leq2.5$
        - $h/b_{fc}\leq6.0$
        - $h/b_{ft}\leq6.0$
          - $\phi V_n=\phi 0.6F_yA_w\left[C_{v2}+\dfrac{1-C_{v2}}{1.15\sqrt{1+(a/h)^2}}\right]
          \;\; \scriptsize\text{ [G2-7]}$

      - Otherwise
        - $\phi V_n=\phi 0.6F_yA_w\left[C_{v2}+\dfrac{1-C_{v2}}{1.15[a/h+\sqrt{1+(a/h)^2}]}\right]
        \;\; \scriptsize\text{ [G2-8]}$  
     
     - b. Check $\Rightarrow \phi V_n > V_u$ <br><br>
       __________________ $>$ _________________ 

### **D. Calculate Shear Capacity for all non-TFA spans**
1. $C_{v1} \;\scriptsize\text{(web shear strength coefficient)}$ __________________ $\;\;$ _________________ 
    - ensure $k_v$ takes $a$ of span into calculation
    - $h/t_w \leq 1.10\sqrt{k_vE/F_y}$
      - $C_{v1}=1.0 
        \;\; \scriptsize\text{ [G2-3]}$

    - $h/t_w > 1.10\sqrt{k_vE/F_y}$
      - $C_{v1}= \dfrac{1.10\sqrt{k_vE/F_y}}{h/t_w}
        \;\; \scriptsize\text{ [G2-4]}$

2. $\phi V_n = \phi 0.6 F_y A_w C_{v1}
    \;\; \scriptsize\text{ [G2-1]}$ __________________ $\;\;$ _________________ 

3. Check $\Rightarrow \phi V_n > V_u$ __________________ $\;\;$ _________________ 

  >## ___*Summary for Shear*___
  > @ ______________ span: $\phi V_n$ ______________ $ft^k > V_u$ ______________ $ft^k\;\;\text{  }$ ___________<br>
  > @ ______________ span: $\phi V_n$ ______________ $ft^k > V_u$ ______________ $ft^k\;\;\text{  }$ ___________<br>
  > @ ______________ span: $\phi V_n$ ______________ $ft^k > V_u$ ______________ $ft^k\;\;\text{  }$ ___________<br>
---

## 4. Bearing Stiffeners Analysis

$\phi = 1.0$

1. Minimum bearing width (if not provided)
    - a. $\phi R_n=R_u=\phi F_{yw}t_w(2.5k+l_b)$ __________________
      1. $k=t_w+ \;\scriptsize\text{weld size (if unknown pick one and adjust later)}$ __________________
  
    - b. $l_b=\dfrac{\phi R_u}{F_{yw}t_w}-2.5k$ __________________
    - c. with knowledge of $l_b$ choose starting bearing length __________________

2. Determine if bearing stiffeners are required
<br>$\large\color{red}\text{I think I have this backwards.... need to think 2.5 vs 5}$

    - a. $\phi R_n=\phi F_{yw}t_w(2.5k+l_b)
       \;\; \scriptsize\text{ [J10-3]}$ __________________

    - b. Check $\Rightarrow \phi R_n > R_u$ <br><br>
   __________________ $>$ _________________ $\scriptsize\text{ (if not true, bearing stiffeners are required.)}$

1. Transverse Stiffener Requirements $\scriptsize\text{ (J10.8)}$
     - a. Check:
       - width of stiffener + $1/2t_w \geq 1/3b_f$  __________________
       - stiffener thickness $\geq 1/2t_f$ __________________
       - extend at least $1/2$ depth of member __________________
  
     - b. Choose plate size to start with __________________
       - ex: 5/8" x 6" each side of member
     - c. Strip of a girder web adds to bearing section __________________
       - $25t_w$ at interior stiffeners
       - $12t_w$ at the end of members

2. Compressive Strength of Bearing Section
   
     - a. $A_g \;\scriptsize\text{(cross sectional area, }in^2)$ __________________
       - $A_g = \sum (l_{st}t_{st}) \;\scriptsize\text{(each stiffener piece and strip of web)}$
  
     - b. $I \;\scriptsize\text{(moment of inertia, }in^4)$ __________________
       - Using parallel axis theorem
         - $I_x = \sum I_{xc} + \sum Ad_y^2$
  
     - c. $r \;\scriptsize\text{(radius of gyration, }in)$ __________________
       - $r=\sqrt{I/A_g}$
  
     - d. $L_c=KL=0.75h \;\scriptsize\text{(effective length, }in)$ __________________
  
       - $\phi = 0.9$
       - $\phi P_n = \phi F_y A_g$ __________________

3. Determine welds needed for connection to girder
  <br> $\scriptsize\text{(using "Shear Flow" Methodology from Salmon/Johnson Book since no AISC provision)}$   

     - a. $r_{uv} \;\scriptsize\text{(capacity needed per weld, }k/in)$ __________________
       - $r_{uv} = \dfrac{R_u}{h(\text{no. of welds})}$
  
     - b. Reference Table J2.4 for minimum weld size __________________
  
     - c. Find actual $\phi r_n$ from table __________________
  
    - d. Check $\Rightarrow \phi r_n > r_u$ <br><br> __________________  $>$ __________________
      - can use intermittent welds
  
4. Check bearing criteria
<br> $\phi =0.75$

     - a. $A_{pb} \;\scriptsize\text{(projected area in bearing, }in^2)$ __________________
       - $A_{pb}= \sum t_{st}(l_{st}-0.75") \\\scriptsize\text{(0.75" approx cutout dimension)}$ 
  
     - b. $\phi R_n = 1.8 F_y A_{pb}
        \;\; \scriptsize\text{ [J7-1]}$ __________________
     - c. Check $\Rightarrow \phi R_n > R_u$ <br><br>
      __________________ $>$ __________________

1. Check need for stiffeners beneath concentrated loads
  <br> $\scriptsize\text{Note: If connections made into Web of Girder, these checks would not be necessary.}$

     - $l_b \;\scriptsize\text{(bearing length from guesstimate, } in.)$ __________________

     - a. Flange Local Buckling $\;\scriptsize\text{[AISC J10.1]}$
        - Assume a compressive force on top of girder
          - $\therefore$ F.L.B. does not apply

     - b. Web Sidesway Buckling $\;\scriptsize\text{[AISC J10.4]}$
        - Girder is laterally braced at concentrated loads
          - $\therefore$ W.S.B. does not apply

     - c. Web Local Yielding $\;\scriptsize\text{[AISC J10.2]}$ <br> $\phi =1.0$
  
        1. $\phi R_n=\phi F_{yw}t_w(5k+l_b) \; \; \large\color{red}\text{this might be 2.5....}
        \;\; \scriptsize\text{ [J10-2]}$ __________________

        1. Check $\Rightarrow \phi R_n > R_u$ <br><br> __________________ $>$ __________________
    - d. Web Local Crippling $\;\scriptsize\text{[AISC J10.3]}$ <br> $\phi =0.75$
      1. Concentrated force to be resisted  __________________
         - distance from member end $\geq d/2$
           - $\phi R_n=\phi 0.8 t_w^2 \left[1+3\left(\dfrac{l_b}{d}\right)\left(\dfrac{t_w}{t_f}\right)^{1.5} \right] \sqrt{\dfrac{EF_{yw}t_f}{t_w}}
          \;\; \scriptsize\text{ [J10-4]}$
      
         - distance from member end $< d/2$
           - $l_b/d \leq 0.2$
             - $\phi R_n=\phi 0.4 t_w^2 \left[1+3\left(\dfrac{l_b}{d}\right)\left(\dfrac{t_w}{t_f}\right)^{1.5} \right] \sqrt{\dfrac{EF_{yw}t_f}{t_w}}
              \;\; \scriptsize\text{ [J10-4]}$

           - $l_b/d > 0.2$
              - $\phi R_n=\phi 0.4 t_w^2 \left[1+\left(\dfrac{4l_b}{d}-0.2\right)\left(\dfrac{t_w}{t_f}\right)^{1.5} \right] \sqrt{\dfrac{EF_{yw}t_f}{t_w}}
            \;\; \scriptsize\text{ [J10-4]}$

      2. Check $\Rightarrow \phi R_n > R_u$ <br><br>
         __________________ $>$ __________________
- If all Checks (a-d) pass $\Rightarrow$ Stiffeners not required beneath concentrated forces
---

## 5. Determine Stiffeners for Tension Field Action Spans

$\;\scriptsize\text{[AISC G2.3]}$

$\large\color{red}\text{notes pages 42- wrong page G3.3 not G2.3}$

A. Transverse Stiffeners, requirements
  1. $\left(\dfrac{b}{t}\right)_{st}\leq 0.56\sqrt{\dfrac{E}{F_{yst}}} \;\;
    \scriptsize\text{ [G2-12]}$ <br><br>
         __________________ $\leq$ __________________

       - probably best to use same material as rest of girder
       - calculate right half, then can chose thickness and determine necessary width
  2. Conservatively, $I_{st} \geq I_{st1}$
       - a. $I_{st}=\dfrac{b_{st}h_{st}^3}{3}$ __________________

       - b. $I_{st1}=\dfrac{h^4 \rho_{st}^{1.3}}{40} \left(\dfrac{F_{yw}}{E} \right)^{1.5}
        \;\; \scriptsize\text{ [G2-14]}$ __________________

         - $\rho_{st}$ = larger of $\mid \dfrac{F_{yw}}{F_{yst}}$ or $1.0$ __________________

B. Connection of Stiffeners to Web & Flange
  1. Shear demand required for weld __________________
       - $f_{nv} = 0.045h\sqrt{\dfrac{F_{yw}^3}{E}}
        ;\; \scriptsize\text{ (S/J p.687)}$
  2. Pick weld sizing for each side __________________
      - can stich weld if excess.

---

## 6. Design Connection B/W Flange and Web

A. Mechanics of Materials Approach __________________
  1. $q= \dfrac{VQ}{I} = \dfrac{VA_y}{I}$

     - Reference AISC Table J2.4 for minimum weld for plate thickness
  2. Pick weld size and find $\phi r_n$ from table __________________
  3. Per AISC E6.2, p. 16.1-38 $\large\color{red}\text{????}$ __________________
       - $Spacing (s_{weld}) \leq 0.75t_w \sqrt{\dfrac{E}{F_y}} \leq 12"$
  4. $L_{weld} \geq 4t_{weld} \; \scriptsize\text{ [J2.2b]}$ __________________
       - round up so length is an reasonable number
  5. Calculate Required Strength
       - a. $\phi R_n  = \text{(qty of welded sides)} \phi r_n L_{weld}$ __________________

       - b. $s_{weld}=\frac{\phi R_n}{q} \;\; in.\;c/c$ __________________
  6. Summarize weld size, length, and spacing ____________________________________
  
B. Verify weld meets requirements for full girder length
  1. Per foot of Girder
       - a. $R_u=q(12")\scriptsize\text{ (Vmax used for q calculation)}$ __________________

       - b. $\phi R_n=\phi r_n L_{weld} (\text{qty of sides welded})(12"/s_{weld})$ __________________

       - c. Check $\Rightarrow \phi R_n > R_u$ <br><br> __________________ $>$ __________________
---

## Provide Design Summary

Sketch to include:
  - identification of each different type of stiffener and location
  - dimensioning of stiffeners/spans
  
Sections to identify the different types of stiffeners and welds